In [3]:
import pandas as pd
import numpy as np
import pulp as pl
from datetime import datetime

# Create sample orders
orders = pd.DataFrame({
    'OrderNr': ['O001', 'O002', 'O003', 'O004', 'O005'],
    'DeliveryDate': ['2024-09-15', '2024-10-10', '2024-09-20', '2024-09-25', '2024-09-30'],
    'OrderStatus': ['Bekreftet', 'Bekreftet', 'Bekreftet', 'Kansellert', 'Bekreftet'],
    'CustomerName': ['AquaGen AS', 'NTNU', 'SalMar Farming AS', 'Ewos Innovation AS', 'Lerøy Midt AS'],
    'Product': ['Gain', 'Shield', 'Gain', 'Shield', 'Gain'],
    'Organic': [False, False, True, False, False],
    'Volume': [500000, 300000, 400000, 200000, 1000000],
    'LockedSite': [None, None, None, None, 'Hønsvikgulen'],
    'PreferredSite': ['Vestseøra', 'Kilavågen Land', 'Bogen', None, None]
})

# Convert delivery date to datetime
orders['DeliveryDate'] = pd.to_datetime(orders['DeliveryDate'])

# Create sample fish groups
fish_groups = pd.DataFrame({
    'Site': ['Vestseøra', 'Kilavågen Land', 'Bogen', 'Hønsvikgulen'],
    'StrippingStartDate': ['2024-08-05', '2024-09-26', '2024-08-05', '2024-07-16'],
    'StrippingStopDate': ['2024-09-02', '2024-11-21', '2024-09-09', '2024-08-27'],
    'Gain-eggs': [7996198, 16451359, 8728493, 16600150],
    'Shield-eggs': [7996198, 16451359, 8728493, 0],  # No Shield eggs at Hønsvikgulen
    'Organic': [True, False, True, False]
})

# Convert dates to datetime
fish_groups['StrippingStartDate'] = pd.to_datetime(fish_groups['StrippingStartDate'])
fish_groups['StrippingStopDate'] = pd.to_datetime(fish_groups['StrippingStopDate'])

def solve_egg_allocation(orders, fish_groups):
    """
    Solve the egg allocation problem using PuLP
    """
    # Filter out cancelled orders
    active_orders = orders[orders['OrderStatus'] != 'Kansellert'].copy()
    active_orders = active_orders.reset_index(drop=True)  # Reset index for consistent access
    
    # Add a dummy fish group for unmatched orders
    dummy_group = pd.DataFrame({
        'Site': ['Dummy'],
        'StrippingStartDate': [pd.to_datetime('2024-01-01')],
        'StrippingStopDate': [pd.to_datetime('2024-12-31')],
        'Gain-eggs': [float('inf')],
        'Shield-eggs': [float('inf')],
        'Organic': [True]
    })
    
    # Reset index of fish_groups before concatenation
    fish_groups_reset = fish_groups.reset_index(drop=True)
    all_fish_groups = pd.concat([fish_groups_reset, dummy_group], ignore_index=True)
    dummy_site_idx = all_fish_groups.index[all_fish_groups['Site'] == 'Dummy'].tolist()[0]
    
    # Create the PuLP problem
    prob = pl.LpProblem("FishEggAllocation", pl.LpMinimize)
    
    # Create binary decision variables for each order-fish group pair
    x = {}
    for i in active_orders.index:
        for j in all_fish_groups.index:
            x[i, j] = pl.LpVariable(f"assign_{i}_{j}", cat='Binary')
    
    # Create penalty variables for each order if it's assigned to dummy group
    dummy_penalties = {}
    for i in active_orders.index:
        dummy_penalties[i] = pl.LpVariable(f"dummy_penalty_{i}", lowBound=0)
    
    # Create penalty variables for not respecting preferred site
    pref_site_penalties = {}
    for i, order in active_orders.iterrows():
        if pd.notna(order['PreferredSite']):
            pref_site_penalties[i] = pl.LpVariable(f"pref_site_penalty_{i}", lowBound=0)
    
    # Objective function: Minimize penalties
    # Priority 1: Avoid using dummy group (highest penalty)
    # Priority 2: Use preferred site when possible (lower penalty)
    prob += (
        1000 * pl.lpSum(dummy_penalties.values()) +
        10 * pl.lpSum(pref_site_penalties.values())
    )
    
    # Constraint 1: Each order must be assigned to exactly one fish group
    for i in active_orders.index:
        prob += pl.lpSum(x[i, j] for j in all_fish_groups.index) == 1
    
    # Constraint 2: Fish group capacity constraints for each product
    for j in all_fish_groups.index:
        if all_fish_groups.loc[j, 'Site'] != 'Dummy':  # Skip the dummy group
            # Gain eggs capacity
            prob += pl.lpSum(
                x[i, j] * active_orders.loc[i, 'Volume'] 
                for i in active_orders.index 
                if active_orders.loc[i, 'Product'] == 'Gain'
            ) <= all_fish_groups.loc[j, 'Gain-eggs']
            
            # Shield eggs capacity
            prob += pl.lpSum(
                x[i, j] * active_orders.loc[i, 'Volume'] 
                for i in active_orders.index 
                if active_orders.loc[i, 'Product'] == 'Shield'
            ) <= all_fish_groups.loc[j, 'Shield-eggs']
    
    # Constraint 3: Organic requirement
    for i in active_orders.index:
        if active_orders.loc[i, 'Organic']:
            for j in all_fish_groups.index:
                if not all_fish_groups.loc[j, 'Organic'] and all_fish_groups.loc[j, 'Site'] != 'Dummy':
                    prob += x[i, j] == 0  # Can't assign organic order to non-organic group
    
    # Constraint 4: Locked site requirement
    for i in active_orders.index:
        if pd.notna(active_orders.loc[i, 'LockedSite']):
            locked_site = active_orders.loc[i, 'LockedSite']
            for j in all_fish_groups.index:
                # Either use the locked site or use dummy group
                if all_fish_groups.loc[j, 'Site'] != locked_site and all_fish_groups.loc[j, 'Site'] != 'Dummy':
                    prob += x[i, j] == 0  # Can't assign to non-locked sites
    
    # Constraint 5: Define dummy penalties
    for i in active_orders.index:
        prob += dummy_penalties[i] >= x[i, dummy_site_idx]
    
    # Constraint 6: Define preferred site penalties
    for i in active_orders.index:
        if pd.notna(active_orders.loc[i, 'PreferredSite']):
            pref_site = active_orders.loc[i, 'PreferredSite']
            # Sum of assignments to non-preferred sites
            non_pref_sites = [j for j in all_fish_groups.index 
                             if all_fish_groups.loc[j, 'Site'] != pref_site 
                             and all_fish_groups.loc[j, 'Site'] != 'Dummy']
            if non_pref_sites:  # Only if there are non-preferred sites
                prob += pref_site_penalties[i] >= pl.lpSum(x[i, j] for j in non_pref_sites)
    
    # Solve the problem
    prob.solve()
    print(f"Status: {pl.LpStatus[prob.status]}")
    
    # Extract results
    results = active_orders.copy()
    results['AssignedGroup'] = None
    results['IsDummy'] = False
    
    for i in results.index:
        for j in all_fish_groups.index:
            if pl.value(x[i, j]) == 1:
                results.loc[i, 'AssignedGroup'] = all_fish_groups.loc[j, 'Site']
                results.loc[i, 'IsDummy'] = (all_fish_groups.loc[j, 'Site'] == 'Dummy')
                break
    
    # Combine with original orders to include cancelled ones
    all_results = orders.copy()
    for i, row in results.iterrows():
        # Find matching index in original orders
        orig_idx = all_results.index[all_results['OrderNr'] == row['OrderNr']][0]
        all_results.loc[orig_idx, 'AssignedGroup'] = row['AssignedGroup']
        all_results.loc[orig_idx, 'IsDummy'] = row['IsDummy']
    
    # Set cancelled orders
    cancelled_idx = all_results[all_results['OrderStatus'] == 'Kansellert'].index
    all_results.loc[cancelled_idx, 'AssignedGroup'] = 'Skipped-Cancelled'
    all_results.loc[cancelled_idx, 'IsDummy'] = False
    
    return all_results

# Run the optimization
result = solve_egg_allocation(orders, fish_groups)

# Display results with more details
print("Final Assignment Results:")
print(result[['OrderNr', 'OrderStatus', 'Product', 'Volume', 'Organic', 'PreferredSite', 'LockedSite', 'AssignedGroup', 'IsDummy']])

# Display remaining capacity in fish groups
def calculate_remaining_capacity(orders, fish_groups):
    remaining = fish_groups.copy()
    
    for j, group in fish_groups.iterrows():
        site = group['Site']
        gain_used = orders[(orders['AssignedGroup'] == site) & (orders['Product'] == 'Gain')]['Volume'].sum()
        shield_used = orders[(orders['AssignedGroup'] == site) & (orders['Product'] == 'Shield')]['Volume'].sum()
        
        remaining.loc[j, 'Gain-eggs-used'] = gain_used
        remaining.loc[j, 'Shield-eggs-used'] = shield_used
        remaining.loc[j, 'Gain-eggs-remaining'] = group['Gain-eggs'] - gain_used
        remaining.loc[j, 'Shield-eggs-remaining'] = group['Shield-eggs'] - shield_used
        
    return remaining

remaining_capacity = calculate_remaining_capacity(result, fish_groups)
print("\nRemaining Capacity:")
print(remaining_capacity[['Site', 'Gain-eggs', 'Gain-eggs-used', 'Gain-eggs-remaining', 
                          'Shield-eggs', 'Shield-eggs-used', 'Shield-eggs-remaining']])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/tikn/miniconda3/envs/dash/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/v6/0qr9kmwj775628yxkv_990840000gn/T/5bba50f082e14c25bda14854ce7c869d-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/v6/0qr9kmwj775628yxkv_990840000gn/T/5bba50f082e14c25bda14854ce7c869d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 29 COLUMNS
At line 138 RHS
At line 163 BOUNDS
At line 184 ENDATA
Problem MODEL has 24 rows, 27 columns and 61 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0002I 5 variables fixed
Cgl0004I processed model has 9 rows, 15 columns (9 integer (9 of which binary)) and 21 elements
Cutoff increment increased from 1e-05 to 9.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 0
